# Real World Application

Integrating our trained LSTM model, we envision a GUI for intuitive drone control via eye movements and blinks. This interface features buttons for commands like Main, Forward, Back, Lift, Land, Right, and Left. It continuously monitors eye gaze, using the LSTM model to interpret eye movements in real time. If a gaze fixes on a button for 3 seconds, the system deems it an intent to select that command. A subsequent blink within a brief window confirms the selection, acting as a "yes" to execute the command. Upon confirmation, the drone receives and performs the action. We utilize denoised EOG data fed into our real-time optimized LSTM model, which runs in a dedicated thread. Every eye gaze fixation is reported to the user through the AsyncTTS class for asynchronous text-to-speech operations.


### 1. Adding required libraries

In [20]:
import threading
from collections import deque
import time
import glob
import serial
import queue 
from queue import Queue
import re
import copy
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
from math import sqrt
import pywt
from scipy import stats
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler, TensorDataset
import tkinter as tk
from tkinter import messagebox
from djitellopy import tello
import pyttsx3

In [21]:
# Set the device to CUDA if available for GPU acceleration; otherwise, use CPU.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [22]:
# Initialize the Tello drone
drone = tello.Tello()


[INFO] tello.py - 129 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.


In [6]:
# Connect to the drone
drone.connect()


[INFO] tello.py - 438 - Send command: 'command'
[INFO] tello.py - 462 - Response command: 'ok'


In [7]:
# Initialize drone battery
print("Battery percentage:", drone.get_battery())

Battery percentage: 94


In [23]:
def read_serial_continuous():
    # Define a pattern to extract horizontal and vertical values from the serial input
    pattern = re.compile(r'H:(\d+), V:(\d+)')  # Regular expression to match the expected format

    # Continuously read from the serial port until an exit condition is met
    while not exit_event.is_set():
        # Check if data is waiting in the serial buffer
        if ser.in_waiting > 0:
            try:
                # Read a line from the serial port, decode it, and strip trailing newlines/spaces
                line = ser.readline().decode('utf-8', errors='ignore').rstrip()
            except UnicodeDecodeError:
                continue  # Ignore lines that cause decoding errors

            # Attempt to match the read line to the expected 'H:###, V:###' format
            match = pattern.match(line)  # Try to match the line to the pattern
            if match:
                 # Extract the horizontal and vertical values from the matched pattern
                hor_part, ver_part = match.groups()
                
                 # Convert the extracted string values to integers
                hor_value_int = int(hor_part)
                ver_value_int = int(ver_part)

                # Ensure the extracted values are within a valid range 
                if 0 <= hor_value_int <= 1023 and 0 <= ver_value_int <= 1023:
                    # Queue the valid data tuple for further processing
                    raw_data_queue.put((hor_value_int, ver_value_int ))


In [24]:
def denoise_signal(signal, wavelet='sym20', level=1):
    """
    Denoise a signal using Discrete Wavelet Transform (DWT) and soft thresholding.
    
    Parameters:
    - signal: The input signal (1D numpy array).
    - wavelet: The type of wavelet to use (e.g., 'db4').
    - level: The level of wavelet decomposition.
    
    Returns:
    - The denoised signal as a 1D numpy array.
    """
    # Decompose to get the wavelet coefficients
    coeff = pywt.wavedec(signal, wavelet, mode='symmetric', level=level)
    
    # Calculate the threshold using the universal threshold method
    sigma = np.median(np.abs(coeff[-1])) / 0.6745
    threshold = sigma * np.sqrt(2 * np.log(len(signal)))
    
    # Apply soft thresholding to remove noise
    coeff_thresh = [pywt.threshold(c, threshold, mode='soft') for c in coeff]
    
    # Reconstruct the signal using the thresholded coefficients
    denoised_signal = pywt.waverec(coeff_thresh, wavelet, mode='symmetric')
    
    return denoised_signal

In [25]:
def denoise_signal_continuous(initial_samples = 50):
    # Initialize lists to accumulate horizontal and vertical data
    accumulated_hor_data = []
    accumulated_ver_data = []

    # Continuously process incoming raw data until a stop condition
    while not exit_event.is_set():
        # Retrieve raw data, normalize it, and accumulate
        hor, ver = raw_data_queue.get() 
        hor = (hor - 512) / 512 # Normalize horizontal data
        ver = (ver - 512) / 512 # Normalize vertical data
        accumulated_hor_data.append(hor)
        accumulated_ver_data.append(ver)

        
        # Denoise data after accumulating a sufficient number of initial samples
        if len(accumulated_hor_data) >= initial_samples:
            # Limit data history to the most recent 'initial_samples' samples
            accumulated_hor_data = accumulated_hor_data[-initial_samples:]
            accumulated_ver_data = accumulated_ver_data[-initial_samples:]

            # Apply denoising to the accumulated data
            denoised_hor_data = denoise_signal(np.array(accumulated_hor_data), wavelet='sym20', level=1)
            denoised_ver_data = denoise_signal(np.array(accumulated_ver_data), wavelet='sym20', level=1)

            # Queue denoised data for further processing
            denoised_hor_queue.put(denoised_hor_data)
            denoised_ver_queue.put(denoised_ver_data)
            
        else:
            # Brief pause to mitigate busy waiting
            time.sleep(0.001)  


In [26]:
def nn_inference_continuous(best_model):
    # Set the buffer size for model inference
    buffer_size = 20  # Last 20 data points are used
    global prev_x_reg, prev_y_reg, blink

    # Continuously perform inference using denoised data    
    while not exit_event.is_set():
        hor_list = denoised_hor_queue.get()
        ver_list = denoised_ver_queue.get()
        
        # Prepare the input for the model
        hor_array = np.array(hor_list)
        ver_array = np.array(ver_list)
        
        # Prepare the input for the model
        combined_array = np.column_stack((hor_array, ver_array))      # Combine horizontal and vertical data

        # Prepare data for model
        model_input = torch.tensor(combined_array[-buffer_size:], dtype=torch.float).unsqueeze(0) 
        model_input = model_input.to(device)
        
        # Inference with no gradient calculation
        with torch.no_grad():
            regression_output, classification_output  = best_model(model_input)
            regression = (regression_output.cpu()).numpy()  # Convert outputs to CPU NumPy array
            blink = (classification_output.cpu()).round()   # Round classification output for blink detection

            # Map regression output to pixel space, clamping to screen bounds            
            x_clamped = max(min(regression[0][0], 1.0), -1.0)
            y_clamped = max(min(regression[0][1], 1.0), -1.0)
            x_pixel =  x_clamped * 540
            y_pixel = - y_clamped * 960 

            # Queue coordinates for visual representation or further processing
            circle_coords_queue.put((round(x_pixel), round(y_pixel)))

In [27]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super(LSTMModel, self).__init__()
        # LSTM layer for sequential data processing
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        # Regression output layers for pointer_x and pointer_y
        self.regressor = nn.Linear(hidden_dim, 2)
        # Classification output layer for blink
        self.classifier = nn.Linear(hidden_dim, 1)
    
    def forward(self, x):
        # Process input through LSTM
        lstm_out, (hn, cn) = self.lstm(x)
        # Get regression values from the last output of LSTM
        regress_output = self.regressor(lstm_out[:, -1, :])
        # Classify blink with sigmoid activation for probability sigmoid activation
        class_output = torch.sigmoid(self.classifier(lstm_out[:, -1, :]))
        
        return regress_output, class_output.squeeze() # Return coordinates and blink probability

In [28]:
# Initialize the model with the same parameters
best_model = LSTMModel(input_dim = 2, hidden_dim = 64, num_layers = 1).to(device)
# Load the state dictionary
model_path = 'best_lstm_model.pth'  
best_model.load_state_dict(torch.load(model_path))
best_model.eval()

LSTMModel(
  (lstm): LSTM(2, 64, batch_first=True)
  (regressor): Linear(in_features=64, out_features=2, bias=True)
  (classifier): Linear(in_features=64, out_features=1, bias=True)
)

In [29]:
def detect_trend_change(data, data_prev):
    # Track current and previous trend directions   
    global trend_prev_char, trend_cur_char
    trend_cur = data - data_prev # Calculate current trend

    # Determine trend direction
    if trend_cur > 0: 
        trend_cur_char = 'positive'
    elif trend_cur < 0:
        trend_cur_char = 'negative'

    # Check for and handle trend change
    if trend_cur_char != trend_prev_char:     
        trend_prev_char = trend_cur_char # Update previous trend
        return data_prev  # Return previous data point if trend changed
    else:
        return 0    # Return 0 if no trend change

In [41]:
def update_circle():
    global current_circle_position, circle_item, nearest_center_x_prev, nearest_center_y_prev, x_integrate, y_integrate, label, label_time_start
    global flag_timer, focus_timer, activation_flag, activation_label
    try:
        # Retrieve new position if available
        new_x, new_y = circle_coords_queue.get_nowait()
    except queue.Empty:
        # Schedule another update if no new position data is available
        root.after(10, update_circle)
        return
        
     # Calculate the nearest grid center to the new position for smooth movement
    nearest_center_x, nearest_center_y = get_nearest_grid_center(new_x, new_y, nearest_center_x_prev, nearest_center_y_prev)

    # Detect trend changes to determine circle movement direction
    x_trend = detect_trend_change(nearest_center_x, nearest_center_x_prev)
    y_trend = detect_trend_change(nearest_center_y, nearest_center_y_prev)

    x_integrate += x_trend
    y_integrate += y_trend

    # Initialize or move the circle based on integrated positions
    if current_circle_position is None:
        # Create the circle on first update
        x_monitor, y_monitor = map_coordinates(x_integrate, y_integrate, mode='monitor')
        circle_item = canvas.create_oval(x_monitor - 100, y_monitor - 100, x_monitor + 100, y_monitor + 100, outline="blue", tags="circle")
        current_circle_position = (x_integrate, y_integrate)
    else:
        # Clamping to screen bounds
        x_integrate = np.clip(x_integrate, -360, 360)  
        y_integrate = np.clip(y_integrate, -760, 760)  
        # Calculate the change in position
        delta_x = x_integrate - current_circle_position[0]
        delta_y = y_integrate - current_circle_position[1]
        # Update circle position with clamped values to ensure it stays within bounds
        canvas.move(circle_item, delta_x, delta_y)  
            
        if blink == 1:
            # Handle blink detection
            blink_message = canvas.create_text(540, 100, text="Blink Detected!", font=("Arial", 30), fill="black")
            # Schedule the blink message to be cleared after 1 seconds
            root.after(1000, lambda: canvas.delete(blink_message))    
            
        # Update the current circle position and check for label changes based on user focus    
        current_circle_position = (x_integrate, y_integrate)
        x_monitor_circle, y_monitor_circle = map_coordinates(x_integrate, y_integrate, mode='monitor')

        # Check for changes in the focused label and reset timer if label changes
        label_prev = label
        label = check_and_print_label_vicinity(x_monitor_circle, y_monitor_circle, label)
        if label_prev != label:
            label_time_start = time.time()
            flag_timer = 1                         # Indicate that the label timer has started
            
        # If the user focuses on a label for more than 3 seconds, initiate activation sequence    
        if (time.time() - label_time_start) >= 3 and label != None and label != "Main":
            if flag_timer == 1:                       # If focus is still on the same label after 3 seconds
                tts.speak(f'Do you want to {label}?') # Ask for confirmation via speech
                focus_timer = time.time()             # Reset the focus timer for potential command activation
                activation_label = label              # Set the label for potential activation
                activation_flag = 1                   # Indicate that the label is ready for activation
            else: 
                if time.time() - focus_timer > 7:   # If no action is taken within 7 seconds
                    label_time_start = time.time()  # Reset label timer
                    tts.speak(f'time up')           # Inform user that the time for activation has passed
                    activation_flag = 0             # Reset activation flag
            flag_timer = 0                          # Reset flag timer after processing
            
        # Activate drone command if the activation flag is set and a blink is detected            
        if activation_flag == 1 and blink == 1:
            tts.speak(f'activating {activation_label}')  # Announce activation
            # Execute drone command based on the activation label
            if activation_label == 'Lift':
                drone.takeoff()
            elif activation_label == 'Land':
                drone.land()
            elif activation_label == 'Right':
                drone.move_right(50)
            elif activation_label == 'Left':
                drone.move_left(50)     
            elif activation_label == 'Forward':
                drone.move_forward(50)
            elif activation_label == 'Backward':
                drone.move_back(50)   
            activation_flag = 0  # Reset activation flag after command execution
            
    # Update previous center coordinates for next iteration        
    nearest_center_x_prev = nearest_center_x
    nearest_center_y_prev = nearest_center_y

    # Schedule the next circle position update
    root.after(1, update_circle)


In [31]:
def check_and_print_label_vicinity(x, y, prev_label):
    # Maps labels to their corresponding screen bounds
    label_bounds = {
        "Main": get_label_bounds("Main"),
        "Forward": get_label_bounds("Forward"),
        "Backward": get_label_bounds("Backward"),
        "Lift": get_label_bounds("Lift"),
        "Land": get_label_bounds("Land"),
        "Left": get_label_bounds("Left"),
        "Right": get_label_bounds("Right")
    }

    # Initialize current_label as None to signify no label is identified initially
    current_label = None  

    # Iterate over label_bounds to find if (x, y) falls within any label's bounds
    for label, bounds in label_bounds.items():
        if is_within_bounds(x, y, bounds):
            current_label = label            # Update current_label upon finding a match
            break                            # Exit loop after the first match to prioritize labels in order
    
    return current_label  # Return the current label

In [32]:
def is_within_bounds(x, y, bounds):
    # Extract the bounding box coordinates
    x1, y1, x2, y2 = bounds
    # Check if the point (x, y) lies within the defined bounds
    return x1 <= x <= x2 and y1 <= y <= y2

In [33]:
def map_coordinates(x, y, mode='cartesian'):
    """
    Translates coordinates between Cartesian and monitor spaces. This is useful for applications
    that require the conversion of geometric positions to screen positions and vice versa.

    Parameters:
    - x, y: The coordinates to map.
    - mode: Specifies the direction of mapping. 'cartesian' converts monitor to Cartesian coordinates,
            while 'monitor' converts Cartesian to monitor coordinates.

    Returns:
    - The mapped coordinates (x_mapped, y_mapped) in the specified space.
    """
    
    # Convert monitor space to Cartesian space
    if mode == 'cartesian':
        # Map from monitor space to Cartesian space
        x_mapped = x - 540  # Shift x to center at 0 (assuming a screen width of 1080px)
        y_mapped = y - 960  # Shift and invert y to center at 0 (assuming a screen height of 1920px)       
    # Convert Cartesian space to monitor space    
    elif mode == 'monitor':
        x_mapped = x + 540  # Shift x to start from the left edge of the screen
        y_mapped = 960 + y  # Invert and shift y to start from the top edge of the screen
    else:
        raise ValueError("Invalid mode specified. Use 'cartesian' or 'monitor'.")
    
    return x_mapped, y_mapped

In [34]:
def get_nearest_grid_center(x, y, prev_center_x = None, prev_center_y = None, border_tolerance_ratio = 0.1):
    # Map input coordinates to monitor space and define grid parameters
    x_monitor, y_monitor = map_coordinates(x, y, mode = 'monitor')
    screen_width = 1080
    screen_height = 1920
    desired_grid_size = 360
    
    # Calculate grid dimensions and cell size
    grid_cols = screen_width // desired_grid_size
    grid_rows = screen_height // desired_grid_size
    cell_width = screen_width / grid_cols
    cell_height = screen_height / grid_rows

    # Identify grid cell of the current point
    col = int(x_monitor // cell_width)
    row = int(y_monitor // cell_height)
    center_x = (col + 0.5) * cell_width
    center_y = (row + 0.5) * cell_height

    # Convert the calculated center back to Cartesian coordinates
    x_cart, y_cart = map_coordinates(center_x, center_y, mode = 'cartesian')

    # Compare with previous center if provided, to determine if snapping back is warranted
    if prev_center_x is not None and prev_center_y is not None:
        # Convert previous centers to monitor coordinates for comparison
        prev_center_x_monitor, prev_center_y_monitor = map_coordinates(prev_center_x, prev_center_y, mode = 'monitor')

        # Calculate distances to previous and new centers
        dist_to_prev_center = ((x_monitor - prev_center_x_monitor) ** 2 + (y_monitor - prev_center_y_monitor) ** 2) ** 0.5
        dist_to_new_center = ((x_monitor - center_x) ** 2 + (y_monitor - center_y) ** 2) ** 0.5
        
        # Calculate border tolerance based on cell size
        border_tolerance = min(cell_width, cell_height) * border_tolerance_ratio

        # Decide whether to return the new center or stick with the previous one
        if dist_to_prev_center < border_tolerance and (dist_to_new_center == 0 or (dist_to_new_center - dist_to_prev_center) / dist_to_new_center < 0.3):
            return prev_center_x, prev_center_y

    return x_cart, y_cart


In [35]:
def get_label_bounds(label_name):
    """
    Determines the bounding box for a given label on the GUI. These boxes are used to map user gaze
    to specific commands. The function defines the size and position of each label's bounding box based on
    the label's name.

    Parameters:
    - label_name: The name of the label for which to get the bounds.

    Returns:
    - A tuple representing the bounding box (x1, y1, x2, y2) of the label, or None if the label is not recognized.
    """
    # Define dimensions and spacing for the bounding boxes
    # Dimensions of the label boxes
    box_width = 200 
    box_height = 50 
    # Spacing between boxes
    vertical_space = 400
    horizontal_space = 400
    # Screen center coordinates
    center_x = 540
    center_y = 960
    
    if label_name == "Main":
        return (center_x - box_width // 2, center_y - box_height // 2, center_x + box_width // 2, center_y + box_height // 2)
    elif label_name == "Forward":
        return (center_x - box_width // 2, center_y - vertical_space - box_height // 2, center_x + box_width // 2, center_y - vertical_space + box_height // 2)
    elif label_name == "Backward":
        return (center_x - box_width // 2, center_y + vertical_space - box_height // 2, center_x + box_width // 2, center_y + vertical_space + box_height // 2)
    elif label_name == "Right":
        return (center_x + horizontal_space - box_width // 2, center_y - box_height // 2, center_x + horizontal_space + box_width // 2, center_y + box_height // 2)
    elif label_name == "Left":
        return (center_x - horizontal_space - box_width // 2, center_y - box_height // 2, center_x - horizontal_space + box_width // 2, center_y + box_height // 2)
    elif label_name == "Lift":
        return (center_x + horizontal_space - box_width // 2, center_y - vertical_space - box_height // 2, center_x + horizontal_space + box_width // 2, center_y - vertical_space + box_height // 2)
    elif label_name == "Land":
        return (center_x - horizontal_space - box_width // 2, center_y + vertical_space - box_height // 2, center_x - horizontal_space + box_width // 2, center_y + vertical_space + box_height // 2)
    else:
        return None  # Return None if label name doesn't match predefined options

In [36]:
# Defines a function to gracefully exit the application.
def quit_app(event = None):
    exit_event.set() # Signal all threads or processes to terminate
    root.destroy()   # Close the GUI window, effectively ending the application

In [37]:
# Combines a frame and a label into a single element.
def create_labeled_frame(master, text, bg_color, fg_color, border_color, border_width, x, y):
    # Create and place the frame at specified coordinates
    frame = tk.Frame(master, width=box_width, height = box_height, bg = bg_color, highlightbackground = border_color, highlightthickness = border_width)
    frame.place(x = x, y = y)
    frame.pack_propagate(False)  # Prevents the frame from resizing to fit the label
    # Create and pack the label inside the frame
    label = tk.Label(frame, text = text, bg = bg_color, fg = fg_color, font = ("Arial", font_size))
    label.pack(expand = True, fill = 'both')
    return frame    # Return the frame containing the label

In [38]:
def start_threads():
    """Function to start all threads."""
    serial_thread.start()     # Begins serial data collection
    denoise_thread.start()    # Starts signal denoising process
    inference_thread.start()  # Initiates neural network inference operations

In [39]:
class AsyncTTS:
    """
    A class for asynchronous text-to-speech operations, enabling non-blocking speech synthesis.
    Utilizes a queue to manage speech requests and a separate thread for processing these requests.
    """
    def __init__(self):
        # Initialize the speech synthesis engine
        self.engine = pyttsx3.init()
        # Queue for holding text-to-speech requests
        self.speech_queue = queue.Queue()
        # Flag to indicate if the queue is currently being processed
        self.is_running = False

    def speak(self, text):
        """
        Adds text to the speech queue and starts processing the queue if not already doing so.
        """
        self.speech_queue.put(text)         # Add text to the queue
        if not self.is_running:
            # Start a new thread to process the queue if one isn't already running
            threading.Thread(target=self._process_queue).start()

    def _process_queue(self):
        """
        Processes the speech queue, synthesizing each text request in sequence.
        Runs on a separate thread to avoid blocking the main application.
        """
        self.is_running = True                   # Indicate processing has started
        while not self.speech_queue.empty():
            text = self.speech_queue.get()       # Retrieve the next text to speak
            self.engine.say(text)                # Pass the text to the speech engine
            self.engine.runAndWait()             # Wait for the speech synthesis to complete
        self.is_running = False                  # Indicate processing is complete

In [40]:
# Initialize an instance of the AsyncTTS class for asynchronous text-to-speech operations.
tts = AsyncTTS()

In [75]:
# Entry point of the application when run 
if __name__ == "__main__":
    try:
        # Initialize global variables and setup the GUI
        current_circle_position, circle_item = None, None
        prev_x_reg, prev_y_reg = 540, 960                    # Default starting position
        trend_prev_char, trend_cur_char = None, None         # Trend tracking
        nearest_center_x_prev, nearest_center_y_prev = 0, 0  # Previous grid center    
        x_integrate, y_integrate = 0, 0                      # Integrated position for smooth movement
        label = None
        flag_timer, activation_flag = 0, 0                   # Timers and flags for label activation
        
        # Initialize serial connection for data acquisition for Linux
        ser = serial.Serial('/dev/ttyACM0',9600)
        
        # Setup queues for thread communication and event for thread termination
        raw_data_queue = Queue()
        denoised_hor_queue = Queue()
        denoised_ver_queue = Queue()
        circle_coords_queue = Queue()
        exit_event = threading.Event()

        # Setup the main GUI window
        root = tk.Tk()
        root.title("Control Panel")
        root.attributes('-fullscreen', True)  # Fullscreen mode
        root.bind('<Escape>', quit_app)       # Escape key binds to quit
         
        # Calculate center positions
        screen_width = root.winfo_screenwidth()        
        screen_height = root.winfo_screenheight()
        center_x = screen_width // 2
        center_y = screen_height // 2

        # Create the main canvas
        canvas = tk.Canvas(root, width=root.winfo_screenwidth(), height=root.winfo_screenheight())
        canvas.pack()
        
        # Customize the size of buttons and labels
        box_width = 200 # Increase box width
        box_height = 50 # Increase box height
        font_size = "30"  # Increase font size
        
        # Create frames with labels using the function
        vertical_space = 400
        horizontal_space = 400
        main_frame = create_labeled_frame(root, "Main", "orange", "black", "red", 5, center_x - box_width // 2, center_y-box_height//2)
        forward_frame = create_labeled_frame(root, "Forward", "orange", "black", "red", 5, center_x - box_width // 2, center_y - vertical_space - box_height // 2)
        backward_frame = create_labeled_frame(root, "Backward", "orange", "black", "red", 5, center_x - box_width // 2, center_y + vertical_space - box_height // 2)
        right_frame = create_labeled_frame(root, "Right", "orange", "black", "red", 5, center_x + horizontal_space - box_width // 2, center_y - box_height // 2)
        left_frame = create_labeled_frame(root, "Left", "orange", "black", "red", 5, center_x - horizontal_space - box_width // 2, center_y - box_height // 2)
        lift_frame = create_labeled_frame(root, "Lift", "orange", "black", "red", 5, center_x + horizontal_space - box_width // 2, center_y - vertical_space - box_height // 2)
        land_frame = create_labeled_frame(root, "Land", "orange", "black", "red", 5, center_x - horizontal_space - box_width // 2, center_y + vertical_space - box_height // 2)
        
        # Quit button
        quit_button = tk.Button(root, text="Quit", command=quit_app)
        quit_button.place(x=0, y=0)  # Position the quit button at the top left corner
        
        # Start threads for handling serial data, denoising, and inference
        serial_thread = threading.Thread(target=read_serial_continuous)
        denoise_thread = threading.Thread(target=denoise_signal_continuous)
        inference_thread = threading.Thread(target=nn_inference_continuous, args=(best_model,))
       
        # Schedule the first tasks and start the GUI event loop
        root.after(1, start_threads)        # Delay thread start to ensure GUI initializes properly
        root.after(100, update_circle)      # Begin updating the circle's position
    
        root.mainloop()                     # Start the GUI

    finally:
        # Cleanup on exit
        ser.close()

[INFO] tello.py - 438 - Send command: 'takeoff'
[INFO] tello.py - 462 - Response takeoff: 'ok'
[INFO] tello.py - 438 - Send command: 'left 50'
[INFO] tello.py - 462 - Response left 50: 'ok'


In [ ]:
    root.destroy()  # Close the GUI window If not close automatically